In [28]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import glob
import cv2

from utils.graphic_utils import *
from utils.detectron_utils import *
from utils.dataframe_utils import *
from utils.matlab_utils import *

## Area covered by Object Representation

In [29]:
################## SET PATHS ##################
PANOPTIC_DATA_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/panoptic-seg/'
PANOPTIC_VIDEO_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/'
GAZE_COM_GAZE_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_data.nosync/LUEBECK_GAZE/GazeCom_ground_truth/'
GAZE_COM_VIDEO_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_data.nosync/LUEBECK/'
PROTO_OBJECT_DIR = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_code/proto-objects/'

data_path_list = [video for video in glob.glob(PANOPTIC_DATA_DIR + '*_data.npy')]
data_path_list.sort()

info_path_list = [video for video in glob.glob(PANOPTIC_DATA_DIR + '*_deepsort.npy')]
info_path_list.sort()

video_path_list = [video for video in glob.glob(GAZE_COM_VIDEO_DIR + '/*')]
video_path_list.sort()

gaze_folder_list = [video for video in glob.glob(GAZE_COM_GAZE_DIR + '/*')]
gaze_folder_list.sort()

In [14]:
#Area Proto Objects
area_covered_by_thing = []
for v in tqdm(range(0,18)):
    panoptic_seg = np.load(data_path_list[v], allow_pickle=True) # change storage.py line 134 for this
    segments_info = np.load(info_path_list[v],allow_pickle=True) 
    thing_area = []
    for frame in (range(len(panoptic_seg))):
        area = 0
        for segment in segments_info[frame]:
            if segment['isthing']:
                area = area + segment['area']
        thing_area.append(area)
    area_covered_by_thing.append(np.mean(thing_area)/(1280*720))

In [30]:
#Area Proto Objects
area_covered_by_proto = []
for v,path in tqdm(enumerate(video_path_list), total = len(video_path_list)):
    video_name = os.path.splitext(os.path.basename(path))[0]
    gaze_dir = gaze_folder_list[v] 
    gaze_path_list = [video for video in glob.glob(gaze_dir + '/*')]
    gaze_path_list.sort()
    ##### Load Proto Object from .mat file #####
    waltherProto, motionProto = load_proto_objects(PROTO_OBJECT_DIR, video_name)
    proto_area = []
    for f in range(len(motionProto)):
        resized = np.zeros((720,1280))
        for n in range(0,5):
            resized += cv2.resize(motionProto[f,:,:,n], (1280,720), interpolation = cv2.INTER_AREA)
        proto_area.append(sum(sum(resized)))
    area_covered_by_proto.append(np.mean(proto_area)/(1280*720))


In [31]:
#Area Low Level Most Salient Objects
lowlevel_salient_objects_path = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/lowlevel-salient-objects/'
area_covered_by_salient_object = []
for v in tqdm(range(0,18)):
    video_name = os.path.splitext(os.path.basename(video_path_list[v]))[0]
    lowlevel_mask_path = lowlevel_salient_objects_path + video_name + '_lowlevel-salient-objects.npy'
    lowlevel_salient_object_ids = np.load(lowlevel_mask_path,allow_pickle=True) # Includes the segment_id of the 5 motion_salient objects per frame
    panoptic_seg = np.load(data_path_list[v], allow_pickle=True) # change storage.py line 134 for this
    segments_info = np.load(info_path_list[v],allow_pickle=True) 
    salient_object_area = []
    for frame in range(min(len(panoptic_seg),len(lowlevel_salient_object_ids))):
        area = 0
        for segment in segments_info[frame]:
            if segment['id'] in lowlevel_salient_object_ids[frame]:
                area = area + segment['area']
        salient_object_area.append(area)
    area_covered_by_salient_object.append(np.mean(salient_object_area)/(1280*720))

In [32]:
#Area Motion Most Salient Objects
lowlevel_salient_objects_path = '/Users/timschroder/Documents/Uni/Bachelorarbeit/project_results.nosync/motion-salient-objects/'
area_covered_by_salient_object_motion = []
for v in tqdm(range(0,18)):
    video_name = os.path.splitext(os.path.basename(video_path_list[v]))[0]
    lowlevel_mask_path = lowlevel_salient_objects_path + video_name + '_motion-salient-objects.npy'
    if os.path.isfile(lowlevel_mask_path):
            
        lowlevel_salient_object_ids = np.load(lowlevel_mask_path,allow_pickle=True) # Includes the segment_id of the 5 motion_salient objects per frame
        panoptic_seg = np.load(data_path_list[v], allow_pickle=True) # change storage.py line 134 for this
        segments_info = np.load(info_path_list[v],allow_pickle=True) 
        salient_object_area = []
        for frame in range(min(len(panoptic_seg),len(lowlevel_salient_object_ids))):
            area = 0
            for segment in segments_info[frame]:
                if segment['id'] in lowlevel_salient_object_ids[frame]:
                    area = area + segment['area']
            salient_object_area.append(area)
        area_covered_by_salient_object_motion.append(np.mean(salient_object_area)/(1280*720))
    else:
        area_covered_by_salient_object_motion.append(0)

In [33]:
path = 'data/df_foveation'
df_foveation = pd.read_pickle(path)

In [34]:
video_names = df_correlation.video.unique()
df_area_things = pd.DataFrame(np.array([(thing_area[v]) for v in range(0,18)]).transpose(), index=thing_classes, columns=video_names)
df_area_stuff.to_pickle('gazecom_area_things_df')
df_area_stuff = pd.DataFrame(np.array([(stuff_area[v]) for v in range(0,18)]).transpose(), index=stuff_classes, columns=video_names)
df_area_stuff.to_pickle('gazecom_area_stuff_df')

NameError: name 'df_correlation' is not defined

In [ ]:
covered_by_thing = np.array([sorted(thing_area[v], reverse=True)[:3] for v in range(0,18)]).transpose()
covered_by_stuff = np.array([sorted(stuff_area[v], reverse=True)[:3] for v in range(0,18)]).transpose()
thing_others = [sum(sorted(thing_area[v], reverse=True)[3:]) for v in range(0,18)]
stuff_others = [sum(sorted(stuff_area[v], reverse=True)[3:]) for v in range(0,18)]
thing_label = np.array([np.array(thing_classes)[(-thing_area[v]).argsort()[:3]] for v in range(0,18)]).transpose()
stuff_label = np.array([np.array(stuff_classes)[(-stuff_area[v]).argsort()[:3]] for v in range(0,18)]).transpose()

In [ ]:
video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({'Top 1 Thing': covered_by_thing[0]/vid_area*100,
                        'Top 2 Thing': covered_by_thing[1]/vid_area*100,
                        'Top 3 Thing': covered_by_thing[2]/vid_area*100,
                        'Other Things': np.array(thing_others)/vid_area*100,
                        #'stuff1': covered_by_stuff[0],
                        #'stuff2': covered_by_stuff[1],
                       # 'stuff3': covered_by_stuff[2],
                       # 'stuff_others': stuff_others,
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)

ax = df_bar.plot(kind='barh', label='Correlation of Gaze with Proto Obejcts', stacked=True, color={"#1B4F72" : "Top 1 Thing", "#21618C" : "Top 2 Thing","#2E86C1" : "Top 3 Thing","#3498DB" : "Other Things"},rot=0, fontsize=12, figsize=(12,8), edgecolor = "white", width=0.7)
plt.legend(loc=1, facecolor='white', framealpha=1, fontsize=14)
vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)

plt.xlabel("Area covered by panoptic 'thing' [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")


In [ ]:
video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({'Top 1 Stuff': covered_by_stuff[0]/vid_area*100,
                        'Top 2 Stuff': covered_by_stuff[1]/vid_area*100,
                        'Top 3 Stuff': covered_by_stuff[2]/vid_area*100,
                        'Other Stuff': np.array(stuff_others)/vid_area*100,
                        #'stuff1': covered_by_stuff[0],
                        #'stuff2': covered_by_stuff[1],
                       # 'stuff3': covered_by_stuff[2],
                       # 'stuff_others': stuff_others,
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh', label='Correlation of Gaze with Proto Obejcts', stacked=True, color={"#943126" : "Top 1 Thing", "#B03A2E" : "Top 2 Thing","#E74C3C" : "Top 3 Thing","#EC7063" : "Other Things"},rot=0, fontsize=12, figsize=(12,8), edgecolor = "white", width=0.7)
plt.legend(loc=2, facecolor='white', framealpha=1, fontsize=14)
vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlim(0, 100)
plt.xlabel("Area covered by panoptic 'stuff' [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")


In [ ]:
video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({'Proto Objects': np.array(area_covered_by_proto)*100,
                        'Panoptic Things': np.array(area_covered_by_thing)*100,
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=1, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Area covered by Object Representations [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
plt.xlim(0,25)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")


In [ ]:
video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({'Proto Objects': df_foveation.groupby(['video']).mean().proto_object.values*100,
                        'Panoptic Things': df_foveation.groupby(['video']).mean().isThing.values*100
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=1, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Correlation of Foveation to Object Representation [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
plt.xlim(0,100)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

In [ ]:
path = 'df_foveation'
df_foveation = pd.read_pickle(path)

video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({'Proto Objects': df_foveation.groupby(['video']).mean().proto_object.values/np.array(area_covered_by_proto),
                        'Panoptic Things': df_foveation.groupby(['video']).mean().isThing.values/np.array(area_covered_by_thing)
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=1, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Correlation of Foveation to Object Representation relative to covered area",labelpad=10, weight='bold', size=14)
plt.tight_layout()
#plt.xlim(0,100)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

## Most Salient Objects

In [ ]:
df_foveation['lowlevel_salient_object_bool'] = np.where(df_foveation['lowlevel_salient_object'].isna(),0,1)
df_foveation['motion_salient_object_bool'] = np.where(df_foveation['motion_salient_object'].isna(),0,1)
video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({
                        'Proto Objects': df_foveation.groupby(['video']).mean().proto_object.values*100,
                        'Low Level Most Salient Objects': df_foveation.groupby(['video']).mean().lowlevel_salient_object_bool.values*100,
                        'Motion Most Salient Objects (only for 3 videos)': df_foveation.groupby(['video']).mean().motion_salient_object_bool.values*100
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=1, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Correlation of Foveation to Most Salient Objects [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
plt.xlim(0,100)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

In [ ]:
df_foveation['lowlevel_salient_object_bool'] = np.where(df_foveation['lowlevel_salient_object'].isna(),0,1)
df_foveation['motion_salient_object_bool'] = np.where(df_foveation['motion_salient_object'].isna(),0,1)
video_names = df_foveation.video.unique()
df_bar = pd.DataFrame({
                        'Proto Objects': df_foveation.groupby(['video']).mean().proto_object.values/area_covered_by_proto,
                        'Low Level Most Salient Objects': df_foveation.groupby(['video']).mean().lowlevel_salient_object_bool.values/area_covered_by_salient_object,
                        'Motion Most Salient Objects (only for 3 videos)': df_foveation.groupby(['video']).mean().motion_salient_object_bool.values/area_covered_by_salient_object_motion
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=4, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Correlation of Foveation to Most Salient Objects relative to covered area",labelpad=10, weight='bold', size=14)
plt.tight_layout()
#plt.xlim(0,100)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")

## Post Saccadic Behaviour

In [ ]:
path = 'data/df_foveation'
df_foveation = pd.read_pickle(path)

In [ ]:
df_post_saccade = df_foveation[df_foveation.event_id == 0].head(10)
unique_event_ids = (np.unique(df_foveation[pd.notna(df_foveation.event_id)].event_id)).astype(int)
for i in tqdm(unique_event_ids):
    df_post_saccade = pd.concat([df_post_saccade, df_foveation[df_foveation.event_id == i].head(10)], ignore_index=True)

In [ ]:
video_names = df_post_saccade.video.unique()
df_bar = pd.DataFrame({'Proto Objects (40ms)': df_post_saccade.groupby(['video']).mean().proto_object.values*100,
                        'Proto Objects': df_foveation.groupby(['video']).mean().proto_object.values*100,
                        'Panoptic Things (40ms)': df_post_saccade.groupby(['video']).mean().isThing.values*100,
                        'Panoptic Things': df_foveation.groupby(['video']).mean().isThing.values*100
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=4, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Correlation of Foveation 40ms after Saccade to Object Representation [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
plt.xlim(0,100)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")
plt.savefig('/Users/timschroder/Documents/Uni/Bachelorarbeit/thesis/images/corr_both_object_rep_40ms.png', dpi=500)

In [ ]:
df_post_saccade.isThing.value_counts(normalize=True) # Correlation of Panoptic Things (40ms)

In [ ]:
df_foveation.isThing.value_counts(normalize=True) # Correlation of Panoptic Things (entire foveation)

## Saccades within Objects

In [ ]:
gazecom_gt_event_df = pd.read_pickle('data/gazecom_gt_event_df')
df_events_foveation = pd.read_pickle('data/df_events_foveation')

In [ ]:
num_vid_saccades = np.zeros(18)
num_vid_saccades_per_sub = np.zeros(18)
v = 0
for video in gazecom_gt_event_df.video.unique():
    dtemp = gazecom_gt_event_df[(gazecom_gt_event_df.event == 2) & (gazecom_gt_event_df.video == video)]
    num_vid_saccades[v] = len(dtemp)
    num_vid_saccades_per_sub[v] = len(dtemp)/len(dtemp.subject.unique())
    v += 1

In [ ]:
track_return = []
for video in tqdm(df_events_foveation.video.unique()):
    single_track_return = []
    for subject in df_events_foveation.subject.unique(): 
        dtemp = df_events_foveation[(df_events_foveation['video']==video)&(df_events_foveation['subject']==subject)]  
        non_unique_tracks = dtemp.panoptic_track.value_counts()[(dtemp.panoptic_track.value_counts().values > 1)].index
        for track_id in non_unique_tracks:
            single_track_return.append(dtemp.event_id[dtemp.panoptic_track == track_id].diff(periods=1).values[1:])
            #track_return.append(single_track_return[1:])
    if (len(single_track_return) != 0):
        track_return.append(np.hstack(single_track_return))
    else: 
        track_return.append(np.nan)


In [ ]:
saccades_within_object = [(np.sum(track_return[i] == 2)/num_vid_saccades[i]) for i in range(0,18)]
saccades_return = [(np.sum(track_return[i] != 2)/num_vid_saccades[i]) for i in range(0,18)]

video_names = df_post_saccade.video.unique()
df_bar = pd.DataFrame({'Saccades within Objects': np.array(saccades_within_object)*100,
                        'Saccades returning to know Objects': np.array(saccades_return)*100
                        }
                        ,index = video_names)
df_bar = df_bar.sort_index(ascending=False)
ax = df_bar.plot(kind='barh',rot=0, fontsize=12, figsize=(12,8), width=0.7)
plt.legend(loc=1, facecolor='white', framealpha=1, fontsize=14)

vals = plt.xticks()
for tick in vals[0]:
    plt.axvline(x=tick, linestyle='dashed', alpha=0.4, color='#eeeeee', zorder=1)
plt.xlabel("Saccades exploring know Objects [%]",labelpad=10, weight='bold', size=14)
plt.tight_layout()
#plt.xlim(0,100)
plt.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on", left="off", right="off", labelleft="on")
plt.savefig('/Users/timschroder/Documents/Uni/Bachelorarbeit/thesis/images/saccades_within_objects.png', dpi=500)

In [ ]:
print('Saccades made within Objects: ', np.mean(saccades_within_object))
print('Saccades returning to know Objects: ', np.mean(saccades_return))